In [1]:
import os
import numpy as np
import torch
import pandas as pd
from glob import glob
from scipy.io import loadmat
import matplotlib.pyplot as plt

In [2]:
os.path.join(os.getcwd(), "src/", "motor_cortex/data/")

'/home/linux-pc/gh/CRCNS/src/motor_cortex/data/'

In [3]:
motion_data_file_l = glob(os.path.join(os.getcwd(), "src/", "motor_cortex/data/data/", "**", "motion*.csv"), recursive=True)
ecog_data_file_l = glob(os.path.join(os.getcwd(), "src/", "motor_cortex/data/data/", "**", "ecog*.csv"), recursive=True)

# Reading only the Left Wrist data and training to detect depending on the Left Wrist

In [4]:
motion_data_file_l

['/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Bilateral/2018-07-12_(S1)/motion_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Bilateral/2018-10-04_(S6)/motion_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Bilateral/2018-09-27_(S5)/motion_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Bilateral/2018-07-19_(S2)/motion_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Bilateral/2018-08-14_(S4)/motion_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Bilateral/2018-07-26_(S3)/motion_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-10_(S7)/motion_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-24_(S10)/motion_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-04-29_(S1)/motion_data.csv',
 '/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-06_(S6)/motion_data.csv',
 '/home/linux-pc/gh/C

In [5]:
motion_data_file_l[0]

'/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Bilateral/2018-07-12_(S1)/motion_data.csv'

In [6]:
motion_data_file_l[6]

'/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-10_(S7)/motion_data.csv'

In [7]:
motion_data_file_l[16]

'/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Contralateral/2018-03-15_(S1)/motion_data.csv'

In [8]:
ecog_data_file_l[0]

'/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Bilateral/2018-07-12_(S1)/ecog_data.csv'

In [9]:
ecog_data_file_l[6]

'/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Ipsilateral/2018-05-10_(S7)/ecog_data.csv'

In [10]:
ecog_data_file_l[16]

'/home/linux-pc/gh/CRCNS/src/motor_cortex/data/data/Contralateral/2018-03-15_(S1)/ecog_data.csv'

## Bilateral Data

In [ ]:
motion_data_file_l[0]

In [ ]:
motion_data_file = motion_data_file_l[0]

In [ ]:
ecog_data_file = ecog_data_file_l[0]

In [ ]:
motion_data_file

In [ ]:
ecog_data_file

In [ ]:
motion_data_bilateral_2018_07_12_S1 = pd.read_csv(motion_data_file)

In [ ]:
motion_data_bilateral_2018_07_12_S1["Motion_time"]

In [ ]:
ecog_data_bilateral_2018_07_12_S1 = pd.read_csv(ecog_data_file)
channel_data = ecog_data_bilateral_2018_07_12_S1.columns[1:-1].values
ecog_data_bilateral_2018_07_12_S1[channel_data].values

In [ ]:
ecog_data_bilateral_2018_07_12_S1["Time"]

In [ ]:
plt.plot(ecog_data_bilateral_2018_07_12_S1["Time"], ecog_data_bilateral_2018_07_12_S1[channel_data].values)

In [ ]:
ecog_channels = ecog_data_bilateral_2018_07_12_S1.columns[1:-1]

In [ ]:
ecog_data_bilateral_2018_07_12_S1[ecog_channels].values

In [ ]:
motion_data_bilateral_2018_07_12_S1_left_wrist = motion_data_bilateral_2018_07_12_S1[motion_data_bilateral_2018_07_12_S1.columns[1:4]]

In [ ]:
motion_data_bilateral_2018_07_12_S1_left_wrist

In [ ]:
motion_data_bilateral_2018_07_12_S1_left_wrist

In [ ]:
ecog_data_bilateral_2018_07_12_S1[channel_data]

In [ ]:
del ecog_data_bilateral_2018_07_12_S1

## Ipsilateral Data (Right Wrist)

In [11]:
ecog_data_ipsilateral = pd.read_csv(ecog_data_file_l[6])

In [12]:
ecog_data_ipsilateral_channel_only = ecog_data_ipsilateral[ecog_data_ipsilateral.columns[1:-1]]

In [13]:
ecog_data_ipsilateral_channel_only

,Channel 1,Channel 2,Channel 3,Channel 4,Channel 5,Channel 6,Channel 7,Channel 8,Channel 9,Channel 10,...,Channel 55,Channel 56,Channel 57,Channel 58,Channel 59,Channel 60,Channel 61,Channel 62,Channel 63,Channel 64
0,-57.40,-150.63,-365.63,201.56,-0.63,-185.73,193.33,144.38,-116.25,-350.94,...,160.42,53.44,138.44,264.90,138.44,31.35,60.73,392.60,102.81,218.13
1,-111.56,-309.17,-438.23,12.60,-115.83,-290.83,49.79,-40.83,-198.65,-474.17,...,149.17,15.73,141.77,177.08,98.33,-4.69,21.77,352.60,60.73,112.50
2,-98.54,-358.33,-509.06,-35.00,-68.65,-333.23,-20.42,-84.90,-218.23,-423.65,...,65.73,1.46,116.35,151.88,77.60,25.52,-60.63,334.06,-22.08,72.60
3,-50.21,-389.06,-505.73,-91.46,-66.98,-299.90,-47.40,-148.23,-295.21,-392.08,...,-6.77,-7.50,37.81,44.06,1.77,38.13,-65.73,302.19,-106.46,36.25
4,-170.73,-483.02,-521.04,-164.27,-110.10,-337.60,-107.81,-243.85,-248.54,-432.81,...,-50.52,-91.46,7.81,-13.54,-48.65,-10.94,-128.02,203.02,-149.48,-16.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939693,1662.60,2705.94,2607.71,2160.10,1632.08,2062.71,2649.58,2356.56,2105.83,2160.10,...,317.81,269.17,220.00,519.27,318.13,-53.13,472.08,347.81,850.21,386.56
939694,2086.04,3413.23,3413.23,2701.98,1893.23,2528.96,3413.23,3218.02,2623.13,2848.33,...,636.35,549.58,665.21,930.42,638.02,-69.17,1000.63,608.02,1551.25,707.29
939695,1973.23,3413.23,3413.23,2788.33,1724.27,2439.17,3413.23,3389.79,2674.38,2990.21,...,810.10,739.69,848.33,1256.35,731.56,-84.90,1252.60,753.02,1966.15,897.40
939696,1495.21,2951.25,3046.88,2151.35,1133.44,1771.25,3413.23,2804.27,2056.25,2352.40,...,896.77,798.13,974.58,1311.56,805.10,22.81,1270.21,778.96,2010.42,976.77


In [14]:
motions_data_ipsilateral = pd.read_csv(motion_data_file_l[6])

In [15]:
motions_data_ipsilateral

,Fsm,Motion_time,Right_Wrist_X,Right_Wrist_Y,Right_Wrist_Z
0,50,0.000,-0.381382,0.572019,-0.300483
1,50,0.020,-0.382611,0.576180,-0.301025
2,50,0.040,-0.387695,0.585411,-0.300829
3,50,0.060,-0.388635,0.582860,-0.301047
4,50,0.080,-0.391379,0.589105,-0.300799
...,...,...,...,...,...
47272,50,945.803,-0.824160,0.799680,-0.304289
47273,50,945.823,-0.829425,0.804616,-0.304512
47274,50,945.843,-0.829667,0.804960,-0.304857
47275,50,945.862,-0.834612,0.807464,-0.305252


In [16]:
ecog_data_ipsilateral_channel_only

,Channel 1,Channel 2,Channel 3,Channel 4,Channel 5,Channel 6,Channel 7,Channel 8,Channel 9,Channel 10,...,Channel 55,Channel 56,Channel 57,Channel 58,Channel 59,Channel 60,Channel 61,Channel 62,Channel 63,Channel 64
0,-57.40,-150.63,-365.63,201.56,-0.63,-185.73,193.33,144.38,-116.25,-350.94,...,160.42,53.44,138.44,264.90,138.44,31.35,60.73,392.60,102.81,218.13
1,-111.56,-309.17,-438.23,12.60,-115.83,-290.83,49.79,-40.83,-198.65,-474.17,...,149.17,15.73,141.77,177.08,98.33,-4.69,21.77,352.60,60.73,112.50
2,-98.54,-358.33,-509.06,-35.00,-68.65,-333.23,-20.42,-84.90,-218.23,-423.65,...,65.73,1.46,116.35,151.88,77.60,25.52,-60.63,334.06,-22.08,72.60
3,-50.21,-389.06,-505.73,-91.46,-66.98,-299.90,-47.40,-148.23,-295.21,-392.08,...,-6.77,-7.50,37.81,44.06,1.77,38.13,-65.73,302.19,-106.46,36.25
4,-170.73,-483.02,-521.04,-164.27,-110.10,-337.60,-107.81,-243.85,-248.54,-432.81,...,-50.52,-91.46,7.81,-13.54,-48.65,-10.94,-128.02,203.02,-149.48,-16.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939693,1662.60,2705.94,2607.71,2160.10,1632.08,2062.71,2649.58,2356.56,2105.83,2160.10,...,317.81,269.17,220.00,519.27,318.13,-53.13,472.08,347.81,850.21,386.56
939694,2086.04,3413.23,3413.23,2701.98,1893.23,2528.96,3413.23,3218.02,2623.13,2848.33,...,636.35,549.58,665.21,930.42,638.02,-69.17,1000.63,608.02,1551.25,707.29
939695,1973.23,3413.23,3413.23,2788.33,1724.27,2439.17,3413.23,3389.79,2674.38,2990.21,...,810.10,739.69,848.33,1256.35,731.56,-84.90,1252.60,753.02,1966.15,897.40
939696,1495.21,2951.25,3046.88,2151.35,1133.44,1771.25,3413.23,2804.27,2056.25,2352.40,...,896.77,798.13,974.58,1311.56,805.10,22.81,1270.21,778.96,2010.42,976.77


### Creating a Pipeline to train the data

In [17]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

In [18]:
ecog_data_ipsilateral_channel_only.shape[0]

939698

In [19]:
motions_data_ipsilateral_values_only = motions_data_ipsilateral[motions_data_ipsilateral.columns[2:]]

In [20]:
motions_data_ipsilateral_values_only.shape[0]

47277

In [21]:
# Constants
ECOG_CHANNELS = 64
WINDOW_SIZE = 20
MOTION_DIMENSIONS = 3

In [22]:
ECOG_SAMPLES = ecog_data_ipsilateral_channel_only.shape[0]
MOTION_SAMPLES = motions_data_ipsilateral_values_only.shape[0]

In [23]:
ecog_data_ipsilateral_900 = ecog_data_ipsilateral[ecog_data_ipsilateral["Time"] <= 900]

In [24]:
ecog_data_ipsilateral_900_channel_data = ecog_data_ipsilateral_900[ecog_data_ipsilateral_900.columns[1:-1]]

In [25]:
motions_data_ipsilateral_900 = motions_data_ipsilateral[motions_data_ipsilateral["Motion_time"] <= 900]

In [26]:
motions_data_ipsilateral_900_data_only = motions_data_ipsilateral_900[motions_data_ipsilateral_900.columns[2:]]

In [32]:
MOTION_SAMPLES = motions_data_ipsilateral_900_data_only.shape[0]

In [33]:
X = np.stack([ecog_data_ipsilateral_900_channel_data[i*WINDOW_SIZE:(i+1)*WINDOW_SIZE] for i in range(MOTION_SAMPLES)])